# 对景区和酒店评论的词语热度计算

## 1 导入所需的库

In [9]:
import pandas as pd 
from datetime import datetime
import jieba
import math

## 2 对景区评论的词语热度计算

### 2.1 导入预处理后的景区评论数据

In [3]:
data = pd.read_csv("./景区评论.csv",encoding='gbk',dtype=str)  # 载入全部数据
data["评论日期"] = [x[:4] for x in data["评论日期"]]           # 将日期统一为只有年份
data["评论日期"]  = data["评论日期"].astype("int")
data

,景区名称,评论日期,评论内容
0,A01,2020,是亲子游的绝佳场所，门票就是有点贵，不过可以接受，爷爷奶奶不放心小朋友也跟上来了，当天我们十...
1,A01,2020,**景区差不多，票价偏贵了。大马戏比较精彩，八点的场次，6点40才能检票进入，我们6点多看看...
2,A01,2020,很有**特色的亲子酒店，房间里的装修很可爱，小孩子特别喜欢，洗漱用品也很有特色，对应的房间还...
3,A01,2020,有园区的工作人员在那，他会主动给你园区里的地图和表演的时间安排，很周到，上接驳车大概也是34...
4,A01,2020,周五逃课跟朋友在广州集合！终于如愿以偿的到达欢乐世界。学生票198 需要出示相关证件（校卡或...
...,...,...,...
58663,A50,2015,还好吧。我们刚刚到瀑布楼遇到一点小意外，打电话到景区办公室要求帮助，景区值班领导马上行动，在...
58664,A50,2015,山高路远，走的很辛苦。景色宜人爬山很累。
58665,A50,2015,环境很好，空气非常棒，很适合全家旅游，特别是避暑
58666,A50,2015,都很方便，价格实惠吧，可以预早就订好。


### 2.2 分词并计算词语热度

In [4]:
for i in range(1,51):
    if i < 10:
        ## 提取单个景区的当前评论和历史评论数据
        # 提取单个景区的所有评论数据
        # data_i 单个景区的所有评论数据
        data_i = data.loc[data["景区名称"]=="A0"+str(i)]  # 提取单个景点的数据
        data_i = data_i[["评论日期","评论内容"]]          # 提取日期和内容
        # data_now 含2020和2021的评论
        year_now = int(data_i.max()[0])                   
        data_now = data_i.loc[(data_i["评论日期"]==year_now) | (data_i["评论日期"]==year_now-1)]  # 景点评论能查询到的最新年份时间及前一年作为当前评论
        # data_pre 之前年份的评论
        data_pre = data_i
        data_pre = data_pre.drop(data_i[(data_pre["评论日期"]==year_now) | (data_pre["评论日期"]==year_now)].index)  # 之前年份的评论作为历史评论

        ## 计算历史词频
        # 对历史评论分词
        # pre_list 将历史评论分词之后的词语列表
        pre_list = list(jieba.cut(" ".join(data_pre["评论内容"]),cut_all=False))  # cut_all=False 精确模式
        # 去除停用词
        # now_clean: 已去除停用词的词语列表
        pre_clean = []  
        with open("stoplist.txt",encoding='utf-8') as file:
            stop_word=[item[:-1] for item in file.readlines()]+['\n',"玩"]  # 导入停用词 stop_list: 停用词列表
            for word in pre_list:
                if word not in stop_word:   # 如果词语列表的词语不属于停用词，则存入word_clean
                    pre_clean.append(word)
        # 计算历史词频
        # pre_counter 历史词频字典
        pre_counter = {}
        for word in pre_clean:
            if word not in pre_counter.keys():  # 该词不在词频字典里
                pre_counter[word] = 1            # 新建该词的键，词频为1
            else:                                 # 该词在词频字典里
                pre_counter[word] += 1           # 该词的词频值+1
                
        ## 计算当前词频
        # 对当前评论分词
        # now_list 将历史评论分词之后的词语列表
        now_list = list(jieba.cut(" ".join(data_now["评论内容"]),cut_all=False))  # cut_all=False 精确模式
        # 去除停用词
        # now_clean: 已去除停用词的词语列表
        now_clean = []  
        with open("stoplist.txt",encoding='utf-8') as file:
            stop_word=[item[:-1] for item in file.readlines()]+['\n',"玩"]  # 导入停用词 stop_list: 停用词列表
            for word in now_list:
                if word not in stop_word:  # 如果词语列表的词语不属于停用词，则存入word_clean
                    now_clean.append(word)
        # 计算当前词频
        # now_counter 当前词频字典
        now_counter = {}
        for word in now_clean:
            if word not in now_counter.keys():   # 该词不在词频字典里
                now_counter[word] = 1             # 新建该词的键，词频为1
            else:                                 # 该词在词频字典里
                now_counter[word] += 1            # 该词的词频值+1
                
        ## 计算冷却系数
        # coldcof 冷却系数
        coldcof = {}
        # hotcof 热度
        hotcof = {}  
        for word in now_clean:
            if word in pre_counter.keys():
                coldcof[word] = math.log((now_counter[word]+1)/(pre_counter[word]))  # 冷却系数的公式表明冷却系数越低，热度越高
        # minhot 热度最小的词
        minhot = coldcof[max(coldcof,key=coldcof.get)]     # 热度最小的词即为冷却系数最大的值
        for word in coldcof.keys():
            hotcof[word] = (minhot - coldcof[word])*1000   # 热度定义为最大冷却系数与该词冷却系数的差值的1000倍
                
        
        ## 根据冷却系数计算热度
        # word_sort 按热度降序排列的词语列表
        word_sort = sorted(hotcof.items(),key=lambda x:x[1],reverse=True)  # 热度按降序排列
        # word_top20 热度前20位的词语
        word_top20 = pd.DataFrame(columns=['评论热词','热度'],data=word_sort[0:20])        
        
        ## 将top20热度的词语保存到csv文件
        word_top20.to_csv("./A0"+str(i)+".csv",index=False,encoding='gbk')
    
    if i >= 10:
        ## 提取单个景区的当前评论和历史评论数据
        # 提取单个景区的所有评论数据
        # data_i 单个景区的所有评论数据
        data_i = data.loc[data["景区名称"]=="A"+str(i)]  # 提取单个景点的数据
        data_i = data_i[["评论日期","评论内容"]]          # 提取日期和内容
        # data_now 含2020和2021的评论
        year_now = int(data_i.max()[0])                   
        data_now = data_i.loc[(data_i["评论日期"]==year_now) | (data_i["评论日期"]==year_now-1)]  # 景点评论能查询到的最新年份时间及前一年作为当前评论
        # data_pre 之前年份的评论
        data_pre = data_i
        data_pre = data_pre.drop(data_i[(data_pre["评论日期"]==year_now) | (data_pre["评论日期"]==year_now)].index)  # 之前年份的评论作为历史评论

        ## 计算历史词频
        # 对历史评论分词
        # pre_list 将历史评论分词之后的词语列表
        pre_list = list(jieba.cut(" ".join(data_pre["评论内容"]),cut_all=False))  # cut_all=False 精确模式
        # 去除停用词
        # now_clean: 已去除停用词的词语列表
        pre_clean = []  
        with open("stoplist.txt",encoding='utf-8') as file:
            stop_word=[item[:-1] for item in file.readlines()]+['\n',"玩"]  # 导入停用词 stop_list: 停用词列表
            for word in pre_list:
                if word not in stop_word:   # 如果词语列表的词语不属于停用词，则存入word_clean
                    pre_clean.append(word)
        # 计算历史词频
        # pre_counter 历史词频字典
        pre_counter = {}
        for word in pre_clean:
            if word not in pre_counter.keys():  # 该词不在词频字典里
                pre_counter[word] = 1            # 新建该词的键，词频为1
            else:                                 # 该词在词频字典里
                pre_counter[word] += 1           # 该词的词频值+1
                
        ## 计算当前词频
        # 对当前评论分词
        # now_list 将历史评论分词之后的词语列表
        now_list = list(jieba.cut(" ".join(data_now["评论内容"]),cut_all=False))  # cut_all=False 精确模式
        # 去除停用词
        # now_clean: 已去除停用词的词语列表
        now_clean = []  
        with open("stoplist.txt",encoding='utf-8') as file:
            stop_word=[item[:-1] for item in file.readlines()]+['\n',"玩"]  # 导入停用词 stop_list: 停用词列表
            for word in now_list:
                if word not in stop_word:  # 如果词语列表的词语不属于停用词，则存入word_clean
                    now_clean.append(word)
        # 计算当前词频
        # now_counter 当前词频字典
        now_counter = {}
        for word in now_clean:
            if word not in now_counter.keys():   # 该词不在词频字典里
                now_counter[word] = 1             # 新建该词的键，词频为1
            else:                                 # 该词在词频字典里
                now_counter[word] += 1            # 该词的词频值+1
                
        ## 计算冷却系数
        # coldcof 冷却系数
        coldcof = {}
        # hotcof 热度
        hotcof = {}  
        for word in now_clean:
            if word in pre_counter.keys():
                coldcof[word] = math.log((now_counter[word]+1)/(pre_counter[word]))  # 冷却系数的公式表明冷却系数越低，热度越高
        # minhot 热度最小的词
        minhot = coldcof[max(coldcof,key=coldcof.get)]     # 热度最小的词即为冷却系数最大的值
        for word in coldcof.keys():
            hotcof[word] = (minhot - coldcof[word])*1000   # 热度定义为最大冷却系数与该词冷却系数的差值的1000倍
                
        
        ## 根据冷却系数计算热度
        # word_sort 按热度降序排列的词语列表
        word_sort = sorted(hotcof.items(),key=lambda x:x[1],reverse=True)  # 热度按降序排列
        # word_top20 热度前20位的词语
        word_top20 = pd.DataFrame(columns=['评论热词','热度'],data=word_sort[0:20])        
        
        ## 将top20热度的词语保存到csv文件
        word_top20.to_csv("./A"+str(i)+".csv",index=False,encoding='gbk')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\86138\AppData\Local\Temp\jieba.cache
Loading model cost 0.687 seconds.
Prefix dict has been built successfully.


## 3 对酒店评论的词语热度计算

### 3.1 导入预处理后的酒店评论数据

In [10]:
data = pd.read_csv("./酒店评论.csv",encoding='gbk',index_col="评论日期")  # 载入全部数据
data.index = pd.to_datetime(data.index)
data

,酒店名称,评论内容,入住房型
评论日期,,,
2020-01-01,H01,酒店很适合家庭出行,标准客房
2020-01-01,H01,升级了房间 延迟退房 很赞,标准客房
2020-01-01,H01,这几年，每年都会来广州，每次都会住**酒店。因为位置好，酒店的性价比也不错，这次给免费升级了...,标准客房
2020-01-01,H01,酒店很好不错,标准客房
2020-01-01,H01,超五星好评,高级客房
...,...,...,...
2020-08-24,H50,酒店是老字号宾馆，虽然有装修过但设施比较陈旧房间空调比较小，卫生间里的墙壁有剥落现象，浴室的...,高级双床房
2020-08-24,H50,本人觉得还好，价格偏高，没有很高档，服务是很好，设施偏老，也不会很大，喜欢楼下的早茶。,高级大床房
2020-08-24,H50,隔音极差，睡眠让人崩溃。,高级大床房


### 3.2 分词并计算词语热度

In [13]:
for i in range(1,51):
    if i < 10:
        ## 提取单个景区的当前评论和历史评论数据
        # data_i 单个景区的所有评论数据
        data_i = data.loc[data["酒店名称"]=="H0"+str(i)]  # 提取单个景区的所有评论数据
        data_i = data_i[["评论内容"]]  # 提取日期和内容
        # data_now 2020年下半年的评论
        data_now = data_i['2020-7-1':'2020-12-31']  # 含有2021年和2020年的评论作为当前评论
        # data_pre 2020年上半年的评论
        data_pre = data_i['2020-1-1':'2020-6-30']   # 之前年份的评论作为历史评论

        ## 计算历史词频
        # 历史评论分词
        # pre_list 将历史评论分词之后的词语列表
        pre_list = list(jieba.cut(" ".join(data_pre["评论内容"]),cut_all=False))  # cut_all=False 精确模式
        # 去除停用词
        # now_clean: 已去除停用词的词语列表
        pre_clean = []  
        with open("stoplist.txt",encoding='utf-8') as file:
            stop_word=[item[:-1] for item in file.readlines()]+['\n',"玩"]  # 导入停用词 stop_list: 停用词列表
            for word in pre_list:
                if word not in stop_word:  # 如果词语列表的词语不属于停用词，则存入word_clean
                    pre_clean.append(word)
        # 计算历史词频
        # pre_counter 历史词频字典
        pre_counter = {}
        for word in pre_clean:
            if word not in pre_counter.keys():   # 该词不在词频字典里
                pre_counter[word] = 1             # 新建该词的键，词频为1
            else:                                 # 该词在词频字典里
                pre_counter[word] += 1            # 该词的词频值+1
                
        ## 计算当前词频
        # 当前评论分词
        # now_list 将历史评论分词之后的词语列表
        now_list = list(jieba.cut(" ".join(data_now["评论内容"]),cut_all=False))  # cut_all=False 精确模式
        # 去除停用词
        # now_clean: 已去除停用词的词语列表
        now_clean = []  
        with open("stoplist.txt",encoding='utf-8') as file:
            stop_word=[item[:-1] for item in file.readlines()]+['\n',"玩"]  # 导入停用词 stop_list: 停用词列表
            for word in now_list:
                if word not in stop_word:  # 如果词语列表的词语不属于停用词，则存入word_clean
                    now_clean.append(word)
        # 计算当前词频
        # now_counter 当前词频字典
        now_counter = {}
        for word in now_clean:
            if word not in now_counter.keys():   # 该词不在词频字典里
                now_counter[word] = 1             # 新建该词的键，词频为1
            else:                                 # 该词在词频字典里
                now_counter[word] += 1            # 该词的词频值+1
                
        ## 计算冷却系数
        # coldcof 冷却系数
        coldcof = {}
        # hotcof 热度
        hotcof = {}  
        for word in now_clean:
            if word in pre_counter.keys():
                coldcof[word] = math.log((now_counter[word]+1)/(pre_counter[word]))  # 冷却系数的公式表明冷却系数越低，热度越高
        # minhot 热度最小的词
        minhot = coldcof[max(coldcof,key=coldcof.get)]     # 热度最小的词即为冷却系数最大的值
        for word in coldcof.keys():
            hotcof[word] = (minhot - coldcof[word])*1000   # 热度定义为最大冷却系数与该词冷却系数的差值的1000倍
                
        
        ## 根据冷却系数计算热度
        # word_sort 按热度降序排列的词语列表
        word_sort = sorted(hotcof.items(),key=lambda x:x[1],reverse=True)  # 热度按降序排列
        # word_top20 热度前20位的词语
        word_top20 = pd.DataFrame(columns=['评论热词','热度'],data=word_sort[0:20])        
        
        ## 将top20热度的词语保存到csv文件
        word_top20.to_csv("./H0"+str(i)+".csv",index=False,encoding='gbk')
    
    if i >= 10:
        ## 提取单个景区的当前评论和历史评论数据
        # data_i 单个景区的所有评论数据
        data_i = data.loc[data["酒店名称"]=="H"+str(i)]  # 提取单个景区的所有评论数据
        data_i = data_i[["评论内容"]]  # 提取日期和内容
        # data_now 2020年下半年的评论
        data_now = data_i['2020-7-1':'2020-12-31']  # 含有2021年和2020年的评论作为当前评论
        # data_pre 2020年上半年的评论
        data_pre = data_i['2020-1-1':'2020-6-30']   # 之前年份的评论作为历史评论

        ## 计算历史词频
        # 历史评论分词
        # pre_list 将历史评论分词之后的词语列表
        pre_list = list(jieba.cut(" ".join(data_pre["评论内容"]),cut_all=False))  # cut_all=False 精确模式
        # 去除停用词
        # now_clean: 已去除停用词的词语列表
        pre_clean = []  
        with open("stoplist.txt",encoding='utf-8') as file:
            stop_word=[item[:-1] for item in file.readlines()]+['\n',"玩"]  # 导入停用词 stop_list: 停用词列表
            for word in pre_list:
                if word not in stop_word:  # 如果词语列表的词语不属于停用词，则存入word_clean
                    pre_clean.append(word)
        # 计算历史词频
        # pre_counter 历史词频字典
        pre_counter = {}
        for word in pre_clean:
            if word not in pre_counter.keys():   # 该词不在词频字典里
                pre_counter[word] = 1             # 新建该词的键，词频为1
            else:                                 # 该词在词频字典里
                pre_counter[word] += 1            # 该词的词频值+1
                
        ## 计算当前词频
        # 当前评论分词
        # now_list 将历史评论分词之后的词语列表
        now_list = list(jieba.cut(" ".join(data_now["评论内容"]),cut_all=False))  # cut_all=False 精确模式
        # 去除停用词
        # now_clean: 已去除停用词的词语列表
        now_clean = []  
        with open("stoplist.txt",encoding='utf-8') as file:
            stop_word=[item[:-1] for item in file.readlines()]+['\n',"玩"]  # 导入停用词 stop_list: 停用词列表
            for word in now_list:
                if word not in stop_word:  # 如果词语列表的词语不属于停用词，则存入word_clean
                    now_clean.append(word)
        # 计算当前词频
        # now_counter 当前词频字典
        now_counter = {}
        for word in now_clean:
            if word not in now_counter.keys():   # 该词不在词频字典里
                now_counter[word] = 1             # 新建该词的键，词频为1
            else:                                 # 该词在词频字典里
                now_counter[word] += 1            # 该词的词频值+1
                
        ## 计算冷却系数
        # coldcof 冷却系数
        coldcof = {}
        # hotcof 热度
        hotcof = {}  
        for word in now_clean:
            if word in pre_counter.keys():
                coldcof[word] = math.log((now_counter[word]+1)/(pre_counter[word]))  # 冷却系数的公式表明冷却系数越低，热度越高
        # minhot 热度最小的词
        minhot = coldcof[max(coldcof,key=coldcof.get)]     # 热度最小的词即为冷却系数最大的值
        for word in coldcof.keys():
            hotcof[word] = (minhot - coldcof[word])*1000   # 热度定义为最大冷却系数与该词冷却系数的差值的1000倍
                
        
        ## 根据冷却系数计算热度
        # word_sort 按热度降序排列的词语列表
        word_sort = sorted(hotcof.items(),key=lambda x:x[1],reverse=True)  # 热度按降序排列
        # word_top20 热度前20位的词语
        word_top20 = pd.DataFrame(columns=['评论热词','热度'],data=word_sort[0:20])        
        
        ## 将top20热度的词语保存到csv文件
        word_top20.to_csv("./H"+str(i)+".csv",index=False,encoding='gbk')